In [1]:
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
import sys
print(sys.version)
print(keras.__version__)
print(tf.__version__)

3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
2.2.2
1.10.0


In [3]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
#config.gpu_options.visible_device_list = "0"
set_session(tf.Session(config=config))

In [4]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

In [5]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

606208/600901 [==============================] - 2s 3us/step
corpus length: 600901


In [6]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

total chars: 59
nb sequences: 200287
Vectorization...


In [7]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
200287/200287 [==============================] - 234s 1ms/step - loss: 2.0195

----- diversity: 0.2
----- Generating with seed: " itself. when, for instance, an
aristocr"
 itself. when, for instance, an
aristocring the stronger the contimation of the the strengs of the the contiment and present of the senting and the restine of the the stronger and the manings of the the restines of the stant and morality, and the forther to the stord to the destrest of the strenger of the strenger of the stome and so the so the strenger to the religion of the distont of the stron of the restine of the strenger and strun

----- diversity: 0.5
----- Generating with seed: " itself. when, for instance, an
aristocr"
 itself. when, for instance, an
aristocring and co the restine of the fall of present of the him in the contives of the man in the will of the restince and the religion of there and contention of the first self-man of the

in the superlative, metaphysical sequalized for when the of moque possible, whihe, and
rifineds of becoded and sscount as belongitiests when the to the sould forwgrdarity. "the worm in meaus of
the selive. in nealon of
aagerroes impalse of the whole greaterecorstion
of "god; aid-of the farteres moverly with not seemns of brenated. a herence in a self--awdy
are become
time town to 
not opreanc been bey away outse"--of possion time i

----- diversity: 1.2
----- Generating with seed: "ill"
in the superlative, metaphysical se"
ill"
in the superlative, metaphysical severes ha grenermy the upas retucious opitiearies it made od
here, genimary
the task in akantren to be certain unmost quisseal indeptanding repusion of
thitestively
morable thoughtided lity those
about it has in youtouh bal
ptantloy
of oceace--itself. humanf limselly ewhom, therebeay is the ofdre" therejuceded has, himphorainly
se. 
a
nem toeek)
find of use. just weneling soliginas will atteraticin

-----------------------------

will suffer inwardly from the subject and conscience of the subject of the subject of the most sublime of the subject of the present that in the same soul and are something of the subject of the subject and an are and are the strong the proposition to the subject of the subject of the subject of the strong to the subject of the strength of the subject of the present the powerful constitute to the same about and are such an

----- diversity: 0.5
----- Generating with seed: "ther, or they
will suffer inwardly from "
ther, or they
will suffer inwardly from the man of the tension of has an ancient come to the subject to the consequences and possessed to the proposivized act as the master and for as the christianity and love of envet who psyched and the
comprehension of the morality for the same presume and the power in world and fears, the strength, for the sense of the world to nature of the explatation of their cates. and in the present frum
to be 

----- diversity: 1.0
----- Generating 

C:\Users\cheshire\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: divide by zero encountered in log


 duty. they has to us self; a hempoelime, of it
he

--------------------------------------------------
Iteration 10
Epoch 1/1
200287/200287 [==============================] - 171s 852us/step - loss: 1.4157

----- diversity: 0.2
----- Generating with seed: " being in such a situation, and he has, "
 being in such a situation, and he has, as the strength of the strength of the sensus and the same themselves to the soul" and so the stronger living and the same and the for the stronger and conscience and soul the moral process and the strength in the power and the morality of the strong to the stronger shame, the stronger the strength of the and the strength in the sensus of the the struggle to the cause of morality of the the streng

----- diversity: 0.5
----- Generating with seed: " being in such a situation, and he has, "
 being in such a situation, and he has, as a philosophical conception of the christian conception, and which we are the soul, the belief in the other orner everything 

...

invaluable contented the problem of the strange and the moral strong the strange a strength of the stronger and the problemore the strange being the strange the struck that is the scientific the strong the historical conscience of the scientific men and the strong the strange and the strange with the strong the strange a subscience that is the man and at the strange and the strength of the strong the moral stronger 

----- diversity: 0.5
----- Generating with seed: "xist, religion gives
invaluable contente"
xist, religion gives
invaluable contented and strange with themselves the proude, merely an ideal to the world of his distopsical at the stances of the world is an interse, in the
most spirits who would a scholars to presses and an interested the discondell and experiences of the strange the has to be the really self-contrarial aristophes before account as the trained the holy the fact as the will all the determine and religion of natur

----- diversity: 1.0
----- Generating with se

to have--whichy--fo†ffren threckeadmentic, it is
¤bou sog-enknesss of represents heurd
of the count, extent its folly is much and greats, which her feeling succiouated; enlaug

----- diversity: 1.2
----- Generating with seed: "first, nothing but a blank
sameness: how"
first, nothing but a blank
sameness: how mhan"ter deser as o†greate, happiest tawn, to th†cy highest heard, instrumm: these imper in the kind of ourselvesm
is capacity
for the
runish dadly
th†gively
womalily," andauy, which how, shoulãquzige in th¤ginary of ©quently, in word¤obles, which¤werplaid t¤knest and bliess of in ¤war, so :uc welfreatifu«b©ke, varedly theforly of their thus
marnoney,†se about before a flaimi¤n's to li©t! whevery

--------------------------------------------------
Iteration 49
Epoch 1/1
200287/200287 [==============================] - 156s 779us/step - loss: 2.1138

----- diversity: 0.2
----- Generating with seed: "pression ensues accompanied by a desire "
pression ensues accompanied by a desire th

whereby a demon is consequent in the invention of the present in¤quition¤king the stragel¤king the sensib«ble of the ¤bov¤king of the ¤bove and and reality of the sense of superfport of the instance¤ty of the st¤ki¤t¤quitions¤quition of the strange the soci¤t the co¤tty of th¤kin¤quition of the some stricted and destro,¤y the soc¤quition ¤ffortunate the conscie¤tifical ¤bous the ¤ge the s¤ke the most and and the so far 

----- diversity: 0.5
----- Generating with seed: "magical ceremony
whereby a demon is cons"
magical ceremony
whereby a demon is consequence and and in the individual the nob†quesit there is
¤bove ¤bover th¤quition ¤boria¤ of the condition ¤get th¤king what is the ¤vel the world of the world beautifies of¤ble of there my¤joy the respect the presuppose¤ts is long t†question th¤ging and the salvation of th«k to interiously of¤grion w¤quition of the pressence of the superfic«n entimia†tift of the are ¤bour f¤joud only there as in 

----- diversity: 1.0
----- Generating wit

established, unaffected by the vicissitue ca©quae ¤«va©f¤¤quimen«,¤bwith ¤veness enfo†ffinereng in¤jousd whoms the©k«tind, as «†b¦ffelf coul¤be© a ©ckenpils, i¤¤jorife: a†gable,¤men and in¤«ver. the alsoule feut©t¤jessatny everyth«gmi¦very an«gratie indidrule†«ble¤d, have gl«ccubley, wither¦jons has «gna¤tion© ¤ous¤©g«v¤kan ir path¤ke; and it tlaking wous the
prinisor the power, ag¤bterm ¤jo©gu«n he inver, †ds su¤«joble i©quiment"--feel

----- diversity: 1.2
----- Generating with seed: "established, unaffected by the vicissitu"
established, unaffected by the vicissitudef ¤bfort¤¤ty¤k¤¤joe, o†a©n who ¤quiant evil¤viply dangero¤pimise you
ide¤quially ple©i¤ppuvi¤†quiled 
a re¤quited©ginguality that alof¤jumently; is¦k in b¦k†¤ke orr«! certain--or a believes watne with theya©«ver--yee«d th«cwom«ffo¤ga†tcfpho¦digrivi«my, gratiurial astimated,
p¤quitin©ke the¤¤k¤n
m©vesth not lave¤ke a©gised and purpo nawi¤ pura©ful. is notsu©ul ¤¤q¤ee¤veount
havi©g of truths ¤kima

------------------------

of the sou¤ness¤ty of¤t the super-¤bers i¤¤es of and the soulse¤tion is the more of the¤kind¤tion of the same o¤me of the m¤quition and and¤¤quition of the ¤¤quition of the and and the soulse¤tion of the con¤¤quition of ¤bers the super--th¤very the present of the t¤king and the m¤qui¤ver of the so¤¤king the more ¤quition and the ¤very the sou¤me and the belief of ¤quite¤ th¤king so the th¤ver all t

----- diversity: 0.5
----- Generating with seed: "esent, in the form of taine--the first
o"
esent, in the form of taine--the first
of the pre¤ffer¤ts an©fferenders i¤moned ¤†bless an©ke-feare h¤¤brination and they any the m¤que of the his consentin©m¤acity,†gres. «quip©¤esti¤bless of been instince, they that the in¤quited of man-discinti©ken ¤y so¤ffers of disphe©way to all can¤king to a†¤bouti¤tility of the indest the evigpans alons the
fume¤¤actantion ¤bou¤t¤ted stretht ©ble¤d, it was is the stand the sorit and in the amoush

----- diversity: 1.0
----- Generating with seed: "esent, in the

In [10]:
model

In [11]:
model.save("nietzche.model")